# Пример агента за резервацију хотела и летова

Ово решење ће вам помоћи да резервишете авионске карте и хотел. Сценарио је путовање из Лондона Хитроу (LHR) 20. фебруара 2024. до Њујорка (JFK) са повратком 27. фебруара 2025. у економској класи, искључиво са Бритиш Ервејзом. Желим да боравим у хотелу Хилтон у Њујорку, молим вас да обезбедите цене за лет и хотел.


# Иницијализација Azure AI Agent Service и добијање информација о конфигурацији из **.env**

### **.env**

Креирајте .env датотеку

**.env** садржи конекциони стринг за Azure AI Agent Service, модел који користи AOAI, као и одговарајући Google API Search сервис API, ENDPOINT, итд.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Назив вашег Azure AI Agent Service модела за распоређивање"

[**NOTE**] Биће вам потребан модел са ограничењем од 100,000 токена по минуту и ограничењем од 600 захтева по минуту.

  Модел можете добити у Azure AI Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Конекциони стринг вашег Azure AI Agent Service пројекта"

  Конекциони стринг пројекта можете добити у прегледу вашег пројекта на AI Foundry Portal екрану.

- **SERPAPI_SEARCH_API_KEY** = "Ваш SERPAPI Search API КЉУЧ"
- **SERPAPI_SEARCH_ENDPOINT** = "Ваш SERPAPI Search Endpoint"

Да бисте добили назив модела за распоређивање и конекциони стринг пројекта за Azure AI Agent Service, потребно је да креирате Azure AI Agent Service. Препоручује се коришћење [овог шаблона](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) за директно креирање. （***Напомена:*** Azure AI Agent Service је тренутно доступан у ограниченим регионима. Препоручује се да се консултујете са [овим линком](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) како бисте подесили регион.)

Агенту је потребан приступ SERPAPI. Препоручује се регистрација путем [овог линка](https://serpapi.com/searches). Након регистрације, можете добити јединствени API КЉУЧ и ENDPOINT.


# Подешавање

Да бисте покренули овај нотебук, потребно је да се уверите да сте инсталирали потребне библиотеке покретањем `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Ваша верзија Semantic Kernel-а треба да буде најмање 1.27.2.


Учитајте подешавања и ресурсе из ваше .env датотеке, молимо вас да се уверите да сте додали своје кључеве и подешавања и креирали локалну .env датотеку.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Пријавите се на Azure

Сада је потребно да се пријавите на Azure. Отворите терминал и покрените следећу команду:

```bash
az login
```

Ова команда ће вас упутити да унесете своје Azure акредитиве, омогућавајући правилно функционисање Azure AI Agent услуге.


# Објашњење:
Ово је променљива која чува API кључ за приступ SERP (страница резултата претраживача) API услузи. API кључ је јединствени идентификатор који се користи за аутентификацију захтева повезаних са вашим налогом.

Сврха: Сврха овог реда је да се API кључ сачува у променљивој како би се користио за аутентификацију захтева ка SERP API услузи. API кључ је неопходан за приступ услузи и извршавање претрага.

Како добити SERP API кључ: Да бисте добили SERP API кључ, следите ове опште кораке на https://serpapi.com (тачни кораци могу варирати у зависности од специфичне SERP API услуге коју користите):

Изаберите SERP API услугу: Постоји неколико SERP API услуга, као што су SerpAPI, Google Custom Search JSON API и друге. Изаберите ону која најбоље одговара вашим потребама.

Региструјте налог: Идите на веб-сајт изабране SERP API услуге и региструјте налог. Можда ће бити потребно да пружите основне информације и потврдите своју имејл адресу.

Креирајте API кључ: Након регистрације, пријавите се на свој налог и идите на секцију API или контролни панел. Потражите опцију за креирање или генерисање новог API кључа.  
Копирајте API кључ у вашу .env датотеку.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Објашњење:
BASE_URL: Ово је променљива која чува основни URL за SERP API крајњу тачку. Име променљиве BASE_URL је конвенција која указује да је овај URL почетна тачка за прављење API захтева.
'https://serpapi.com/search':

Ово је стварни URL стринг додељен променљивој BASE_URL. Он представља крајњу тачку за извршавање претрага користећи SERP API.

# Сврха:
Сврха овог реда је да дефинише константу која садржи основни URL за SERP API. Овај URL ће се користити као почетна тачка за конструисање API захтева ради извршавања операција претраге.

# Употреба:
Дефинисањем основног URL-а у променљивој, можете га лако поново користити кроз ваш код кад год је потребно да направите захтеве ка SERP API-ју. Ово чини ваш код лакшим за одржавање и смањује ризик од грешака које настају услед ручног уноса URL-а на више места. Тренутни пример је https://serpapi.com/search?engine=bing који користи Bing API за претрагу. Различити API могу се изабрати на https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Објашњење:

Овде се налази ваш код за додатак.

Дефиниција класе: `class BookingPlugin`: Дефинише класу под именом BookingPlugin која садржи методе за резервацију хотела и летова.

Метод за резервацију хотела:

- `@kernel_function(description="booking hotel")`: Декоратор који описује функцију као kernel функцију за резервацију хотела.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Дефинише метод за резервацију хотела са анотираним параметрима и типом повратне вредности.

Метод конструише речник параметара за захтев за резервацију хотела и шаље GET захтев SERP API-ју. Проверава статус одговора и враћа информације о хотелу ако је успешно, или None ако је захтев неуспешан.

Метод за резервацију летова:

- `@kernel_function(description="booking flight")`: Декоратор који описује функцију као kernel функцију за резервацију летова.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Дефинише метод за резервацију летова са анотираним параметрима и типом повратне вредности.

Метод конструише речнике параметара за захтеве за одлазне и повратне летове и шаље GET захтеве SERP API-ју. Проверава статус одговора и додаје информације о лету у резултатски стринг ако је успешно, или исписује поруку о грешци ако је захтев неуспешан. Метод враћа резултатски стринг који садржи информације о лету.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Објашњење:
Увоз модула: Увозе се неопходни модули за Azure акредитиве, AI агента, садржај порука у чету, улогу аутора и декоратор функције језгра.

Асинхрони контекст менаџер: `async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,)`: Ово поставља асинхрони контекст менаџер за руковање Azure акредитивима и креирање клијента за AI агента.

Име агента и инструкције:
- `AGENT_NAME = "BookingAgent"`: Дефинише име агента.
- `AGENT_INSTRUCTIONS = """..."""`: Пружа детаљна упутства агенту о томе како да обрађује захтеве за резервацију.

Креирање дефиниције агента: `agent_definition = await client.agents.create_agent(...)`: Креира дефиницију агента са одређеним моделом, именом и инструкцијама.

Креирање AzureAI агента: `agent = AzureAIAgent(...)`: Креира AzureAI агента користећи клијента, дефиницију агента и дефинисани додатак.

Креирање нити: `thread: AzureAIAgentThread | None = None`: Креира нит за агента. Није неопходно прво креирати нит - ако је вредност `None`, нова нит ће бити креирана током прве инвокације и враћена као део одговора.

Кориснички уноси: `user_inputs = ["..."]`: Дефинише листу корисничких уноса које агент треба да обради.

У блоку `finally`, обришите нит и агента како бисте ослободили ресурсе.


# Аутентикација

Класа `DefaultAzureCredential` је део Azure SDK-а за Python. Она пружа подразумевани начин за аутентикацију са Azure услугама. Покушава да се аутентикује користећи више метода у одређеном редоследу, као што су променљиве окружења, управљани идентитет и Azure CLI акредитиве.

Асинхроне операције: Модул aio указује да класа DefaultAzureCredential подржава асинхроне операције. То значи да је можете користити са asyncio за извршавање захтева за аутентикацију без блокирања.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да превод буде тачан, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
